In [ ]:
from graphframes import *
import networkx as nx
import matplotlib.pyplot as plt
import time
import json
import pandas as pd
import re
from datetime import datetime
import subprocess
global start
from pyspark.sql import SparkSession, Row
import os
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional
import uvicorn
import nest_asyncio

# Sử dụng nest_asyncio để khắc phục vấn đề vòng lặp sự kiện
nest_asyncio.apply()


letters = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'a', 'á', 'à', 'ả', 'ã', 'ạ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ',
    'b', 'c', 'd', 'đ', 'e', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'f', 'g', 'h', 'i', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'j', 'k', 'l', 'm', 'n', 'o', 'ó', 'ò', 'ỏ', 'õ', 'ọ',
    'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'p', 'q', 'r', 's', 't', 'u',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'v', 'w', 'x', 'y', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'z'
]


In [ ]:
df = pd.read_excel('video_list.xlsx')


In [43]:
def find_letter1(word,dsts):   
    check = dsts
    for i,value in check:  
        if value == word:
            row = vertices[i]
            return row
    return None   

def find1(word):
    length = len(word)
    row = vertices[letters.index(word[:1])]
    dsts = eval(row[1][1]) 
    for i in range(2,length+1):
        char = word[:i]
        row = find_letter1(char,dsts)
        if not row:
            return None   
        dsts = eval(row[1][1]) 
    list_link = eval(row[1][2]) 
    return list_link

#######đây là hàm sẽ tìm từng chữ cái 1 trong 1 từ với dsts 
#######là định hướng id cho trước
def find_letter2(word,dsts):   
    check = dsts
    for i,value in check:  
        if value == word:
            row = data_vertices.lookup(i)
            return row
    return None  
#######đây là hàm tìm kiếm từ trong cây bằng cách gọi đến find_letter
def find2(word):
    length = len(word)
    row = data_vertices.lookup(letters.index(word[:1]))
    dsts = eval(row[0][1]) 
    for i in range(2,length+1):
        char = word[:i]
        row = find_letter2(char,dsts)
        if not row:
            return None   
        dsts = eval(row[0][1]) 
    list_link = eval(row[0][2]) 
    return list_link
####hàm đưa ra các đoạn văn có cụm từ đưa vào
###với list là các vị trí tìm kiếm của các từ trong cụm từ
def find_matching_ids_across_multiple_lists(lists):
    matching_ids = set()

    # Tạo một từ điển từ mỗi danh sách để tra cứu nhanh
    dicts = [{item[0]: item[1] for item in lst} for lst in lists]
    # Lấy các id chung giữa tất cả các danh sách
    common_ids = set(dicts[0].keys())
    for d in dicts[1:]:
        common_ids &= set(d.keys())

    # Kiểm tra vị trí liên tiếp giữa tất cả các danh sách
    for common_id in common_ids:
        position_lists = [d[common_id] for d in dicts]
        
        # Kiểm tra nếu có vị trí liên tiếp giữa các danh sách
        if all(any(pos2 - pos1 == 1 for pos1 in position_lists[i] for pos2 in position_lists[i+1]) for i in range(len(position_lists) - 1)):
            matching_ids.add(common_id)
    
    return list(matching_ids)

#tìm kiếm các đoạn văn có chung cụm từ
def find_common_phrase(phrase):
    words = phrase.split()  # Biến để lưu trữ các ID chung
    lists =[]
    for word in words: 
        word = word.lower()
        link = find1(word) 
        if not link:
            return None# Tìm kiếm với từ hiện tại
        lists.append(link)
    match_id = None    
    if lists:
        match_id = find_matching_ids_across_multiple_lists(lists)
    return match_id
#tìm kiếm các text có chung cả đoạn văn
def find_common_paragraph(paragraph):
    phrases = extract_phrases(paragraph) 
    common_links = None 

    for phrase in phrases: 
        links = find_common_phrase(phrase)
        if links:
            if common_links is None:
                common_links = set(links)  # Nếu là từ đầu tiên, khởi tạo tập hợp
            else:
                common_links &= set(links)  # Tìm giao với các kết quả trước đó
        else:
            return []
        # Nếu không còn ID chung nào, có thể dừng sớm
        if not common_links:
            break

    return list(common_links) if common_links else []

#Tách đoạn văn thành các phrase bằng dấu phẩy
def extract_phrases(paragraph):
    # Tách đoạn văn thành các phrase bằng dấu phẩy

    phrases = paragraph.split(',')
    
    # Loại bỏ khoảng trắng thừa ở đầu và cuối mỗi phrase
    phrases = [phrase.strip() for phrase in phrases]
    
    return phrases
    
#chuyển rdd thành 1 đoạn text
def hdfs_to_text(file_path):
    # Đọc tệp văn bản vào một RDD
    rdd = sc.textFile(file_path)

    # Thu thập nội dung RDD thành một danh sách
    txt = " ".join(rdd.collect())
   
    return txt

# name  = /2024-08-27/vertices
def loadTreeData(name):
    # Lấy ngày giờ hiện tại

    # Đường dẫn HDFS để đọc tệp
    link = "/save"
    hdfs_path = link  + name
    print(hdfs_path)
    # Đọc dữ liệu từ HDFS thành RDD
    rdd = sc.textFile(hdfs_path)

    # Chuyển đổi các chuỗi trở lại thành tuple
    rdd = rdd.map(lambda x: eval(x))
    
    print("Dữ liệu đã được đọc và chuyển đổi thành công.")
    return rdd
###########################
def main_search(text):
    a = time.time()
    common_paragraph = find_common_paragraph(text)

    # Lọc các hàng trong dataframe tương ứng với các id trong common_paragraph
    filtered_df = df[df['ID'].isin(common_paragraph)]

    # Chuyển kết quả sang dạng JSON
    result = filtered_df.to_dict(orient='records')
    now = time.time()
    # Chuyển đổi số giây thành đối tượng datetime
    now_dt = datetime.fromtimestamp(now)

    # Định dạng thành chuỗi theo định dạng giờ:phút:giây ngày-tháng-năm
    formatted_time = now_dt.strftime("%H:%M:%S %d-%m-%Y")
    time_load = now-a
    print(time_load,'result',result)
    return {
        "result": result,
        "time_load": time_load,
        "time_search": formatted_time
    }
def get_subtitles(file_id):
    file_path = f'/home/master/Downloads/Data/{file_id}.txt'
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()
        return file_content
    except FileNotFoundError:
        print(f"File tại {file_path} không tồn tại.")
        return None
    except IOError as e:
        print(f"Lỗi đọc file: {e}")
        return None

In [49]:
name  = '/2024-09-03/vertices_v8/vertices'
data_vertices = loadTreeData(name)
vertices = data_vertices.collect()
print(len(vertices))
global data_vertices,vertices

/save/2024-09-03/vertices_v8/vertices
Dữ liệu đã được đọc và chuyển đổi thành công.
102845


In [ ]:
# Đoạn văn cần tìm
paragraph = "đối tượng"

# Tìm các id tương ứng với đoạn văn
a = time.time()
result = main_search(paragraph)

# In kết quả ra màn hình hoặc lưu vào file
print(result)

In [ ]:
# Tạo lớp mô hình cho dữ liệu đầu vào
class ParagraphRequest(BaseModel):
    paragraph: str


# Khởi tạo FastAPI
app = FastAPI()

# Tạo endpoint cho API
@app.post("/get_main_search")
async def get_main_search(request: ParagraphRequest):

    result = main_search(request.paragraph)
    return result  # Trả về JSON trực tiếp
# Khởi chạy FastAPI trong Jupyter Notebook

# Tạo endpoint mới cho API get_subtitles
@app.post("/get_subtitle_by_id")
async def get_subtitle_by_id(id: int):
    result = get_subtitles(id)
    return {"subtitle": result}

In [ ]:
uvicorn.run(app, host="127.0.0.1", port=8080)

INFO:     Started server process [1665347]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)


2.78867244720459 result [{'ID': 725, 'Video ID': 'tPjTTstx5gU', 'Video Title': 'Bản tin trưa 9/3: Toàn cảnh Houthi phóng hơn 400 quả tên lửa tấn công tàu trên Biển Đỏ và Biển Arab', 'Publish Time': '2024-03-09T04:04:55Z', 'Channel': 'Tin24h'}, {'ID': 1379, 'Video ID': '13hQgnu9NyM', 'Video Title': 'Tin Mới Nhất Việt Nam sẵn sàng giúp Thổ Nhĩ Kỳ tái thiết vùng bị động đất', 'Publish Time': '2023-02-17T14:40:21Z', 'Channel': 'Tin24h'}, {'ID': 2826, 'Video ID': 'dQeeHD52OtA', 'Video Title': 'Bản tin tối 14/7: Nghi phạm bắn ông Trump là người bên Đảng Cộng hòa, từng có thành tích học tập tốt', 'Publish Time': '2024-07-14T12:00:07Z', 'Channel': 'Tin24h'}, {'ID': 2860, 'Video ID': '0omroI3qWhU', 'Video Title': 'Tổng thống Putin thăm cấp nhà nước Việt Nam ngày 19 và 20-6 | Tin tức 24h mới nhất', 'Publish Time': '2024-06-17T16:00:07Z', 'Channel': 'Tin24h'}, {'ID': 3526, 'Video ID': 'dgrGaUry9es', 'Video Title': 'Tin tức 24h.Tin Tối 14/8: Tướng Nga tiết lộ người chỉ đạo chiến dịch tấn công của 

INFO:     27.67.99.9:0 - "POST /get_subtitle_by_id?id=3526 HTTP/1.1" 200 OK


Task exception was never retrieved
future: <Task finished name='Task-17' coro=<Server.serve() done, defined at /home/master/.local/lib/python3.8/site-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/home/master/.local/lib/python3.8/site-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/home/master/.local/lib/python3.8/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/home/master/.local/lib/python3.8/site-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/home/master/.local/lib/python3.8/site-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/home/master/.local/lib/python3.8/site-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/l

2.9893901348114014 result [{'ID': 48219, 'Video ID': 'BOKjkl5e9Wo', 'Video Title': 'Tổng thống Hoa Kỳ Joe Biden bắt đầu chuyến thăm cấp Nhà nước tới Việt Nam | VTC Now', 'Publish Time': '2023-09-10T09:00:10Z', 'Channel': 'VTC NOW'}, {'ID': 56493, 'Video ID': 'Ng_WV-LGkG0', 'Video Title': '[Xu hướng 24h] Bà Đặng Thùy Trang: Sẽ không bỏ qua nếu Thùy Tiên tiếp tục “gây chiến” | VTC Now', 'Publish Time': '2022-11-09T05:34:33Z', 'Channel': 'VTC NOW'}]
INFO:     27.67.99.9:0 - "POST /get_main_search HTTP/1.1" 200 OK
INFO:     27.67.99.9:0 - "POST /get_subtitle_by_id?id=48219 HTTP/1.1" 200 OK
2.1050868034362793 result [{'ID': 345, 'Video ID': 'a0uLSrGnBqI', 'Video Title': 'Tin tức | Chào buổi sáng | Tin tức Việt Nam mới nhất 25/2: Miền Bắc liên tiếp đón không khí lạnh', 'Publish Time': '2024-02-24T22:30:11Z', 'Channel': 'Tin24h'}, {'ID': 548, 'Video ID': '3jBF2Sd3hU8', 'Video Title': 'Toàn cảnh hiện trường vụ xe khách chở hàng chục cán bộ hưu trí Bắc Ninh lao xuống vực ở Tam Đảo.THVN', 'Publi

In [ ]:
# def find_letter1(word,dsts):   
#     check = dsts
#     for i,value in check:  
#         print('test ',(i,value))
#         if value == word:
#             row = vertices[i]
#             print('found',row)
#             return row
#     return None   

# def find1(word):
#     length = len(word)
#     row = vertices[letters.index(word[:1])]
#     print('stage1',row)
#     dsts = eval(row[1][1]) 
#     for i in range(2,length+1):
#         char = word[:i]
#         print('stage',i,char)
#         row = find_letter1(char,dsts)
#         if not row:
#             return None   
#         dsts = eval(row[1][1]) 
#     list_link = eval(row[1][2]) 
#     return list_link

# #######đây là hàm sẽ tìm từng chữ cái 1 trong 1 từ với dsts 
# #######là định hướng id cho trước
# def find_letter2(word,dsts):   
#     check = dsts
#     for i,value in check:  
#         print('test ',(i,value),word)
#         if value == word:
#             row = data_vertices.lookup(i)
#             print('found',row)
#             return row
#     return None  
# #######đây là hàm tìm kiếm từ trong cây bằng cách gọi đến find_letter
# def find2(word):
#     length = len(word)
#     row = data_vertices.lookup(letters.index(word[:1]))
#     print('stage1',row)
#     dsts = eval(row[0][1]) 
#     for i in range(2,length+1):
#         char = word[:i]
#         print('stage',i,char)
#         row = find_letter2(char,dsts)
#         if not row:
#             return None   
#         dsts = eval(row[0][1]) 
#     list_link = eval(row[0][2]) 
#     return list_link


# #tìm kiếm các đoạn văn có chung cụm từ
# def find_common_phrase(phrase):
#     words = phrase.split()  # Biến để lưu trữ các ID chung
#     lists =[]
#     for word in words: 
#         print('word ',word )
#         word = word.lower()
#         link = find1(word) 
#         if not link:
#             return None# Tìm kiếm với từ hiện tại
#         lists.append(link)
#     match_id = None    
#     if lists:
#         match_id = find_matching_ids_across_multiple_lists(lists)
#     return match_id
# #tìm kiếm các text có chung cả đoạn văn
# def find_common_paragraph(paragraph):
#     phrases = extract_phrases(paragraph) 
#     common_links = None 

#     for phrase in phrases: 
#         print('phrases ',phrase )
#         links = find_common_phrase(phrase)
#         if links:
#             if common_links is None:
#                 common_links = set(links)  # Nếu là từ đầu tiên, khởi tạo tập hợp
#             else:
#                 common_links &= set(links)  # Tìm giao với các kết quả trước đó
#         else:
#             return []
#         # Nếu không còn ID chung nào, có thể dừng sớm
#         if not common_links:
#             break

#     return list(common_links) if common_links else []

